In [15]:
import pandas as pd
data=pd.read_csv('C:\\Users\\sh921\\Desktop\\SteamComment_preprocessed.csv')

data.head()

,date,text,GameName
0,2020-11-05,재밌는 게임이 핵쟁이 새끼들 때문에 최단기 퇴물이라는 비운의 타이틀을 얻게 된 게임...,APEX LEGENDS
1,2020-12-21,라이브러리 우클릭 입력하면 게임 시작시 소리 사라집니다,APEX LEGENDS
2,2021-09-06,서버 관리 안하냐,APEX LEGENDS
3,2020-12-03,겜 킬때 뜨는 화면 없애는 법을 찾고 광명을 찾앗습니다 감사합니다,APEX LEGENDS
4,2021-10-18,이 게임에서 중국욕 하면 영구정지먹음 이메일로 정지관련 메일을 받았는데 6월30일 ...,APEX LEGENDS


In [16]:
from collections import Counter
#from tensorflow import keras
#from tensorflow.keras.preprocessing.text import Tokenizer

import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.models.wrappers.ldamallet import LdaMallet

In [17]:
import numpy as np
#데이터 프레임의 'text' 열의 값들을 str 형식으로 바꾸기
data.text = data.text.astype(str)


#text와 timestamp 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
data.drop_duplicates(subset=['date','text','GameName'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
data['text'] = data['text'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
data = data.replace({'': np.nan})
data = data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
data = data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(data.isnull().values.any()) 

False


In [28]:
data

,date,text,GameName
0,2020-11-05,재밌는 게임이 핵쟁이 새끼들 때문에 최단기 퇴물이라는 비운의 타이틀을 얻게 된 게임...,APEX LEGENDS
1,2020-12-21,라이브러리 우클릭 입력하면 게임 시작시 소리 사라집니다,APEX LEGENDS
2,2021-09-06,서버 관리 안하냐,APEX LEGENDS
3,2020-12-03,겜 킬때 뜨는 화면 없애는 법을 찾고 광명을 찾앗습니다 감사합니다,APEX LEGENDS
4,2021-10-18,이 게임에서 중국욕 하면 영구정지먹음 이메일로 정지관련 메일을 받았는데 월 일 ...,APEX LEGENDS
...,...,...,...
56798,2012-05-20,벙어리 체험 게임입니다 이 시대에 착취 되지 않을려면 의견제시는 필수입니다,콜 오브 듀티: 모던 워페어
56799,2012-03-31,말이 필요없음 재밋었 멀티에 핑만 빼고,콜 오브 듀티: 모던 워페어
56800,2013-06-10,콜옵계의 가장 명작이라면 이것이 아닐까,콜 오브 듀티: 모던 워페어
56801,2011-09-24,내 인생을 바꾼 최고의 게임이다 콜 오브 듀티 시리즈 중 가장 뛰어나며 스토리도 가...,콜 오브 듀티: 모던 워페어


In [19]:
Data_list=data.text.values.tolist()
#(트윗 하나)씩 가져와서 명사만 추출한 후 리스트로 만들기
data_word=[]
for i in range(len(Data_list)):
    try:
        data_word.append(okt.nouns(Data_list[i]))
    except Exception as e:
        continue


#명사추출이 부자연스러운 경우 사전에 명사 추가

In [29]:
data_word

[['게임',
  '핵',
  '새끼',
  '때문',
  '최',
  '단기',
  '퇴',
  '물이',
  '비운',
  '타이틀',
  '게임',
  '에이',
  '팔',
  '레전드',
  '스팀',
  '핵',
  '새끼',
  '무엇',
  '스팀',
  '실행',
  '오리진',
  '런처',
  '이중',
  '실행',
  '안',
  '해도',
  '오리진',
  '계정',
  '연동',
  '게임',
  '실행',
  '게',
  '최고',
  '장점'],
 ['라이브러리', '클릭', '입력', '게임', '작시', '소리'],
 ['서버', '관리'],
 ['겜', '킬때', '화면', '법', '광명'],
 ['이',
  '게임',
  '중국',
  '욕',
  '영구',
  '정지',
  '먹음',
  '이메일',
  '정지',
  '관련',
  '메일',
  '월',
  '일',
  '일주일',
  '정지',
  '불구',
  '난',
  '월',
  '일',
  '게임',
  '못',
  '메일',
  '문의',
  '번',
  '답',
  '쓰기',
  '겜',
  '아래',
  '메일',
  '전문',
  '임'],
 ['아아', '이', '감각', '년'],
 ['우리', '파티', '전투력', '파티', '때', '게임'],
 ['서버', '병신', '겜', '한국인', '기본', '핑때문', '핸디캡', '게임', '재미', '시발', '재미'],
 ['영정'],
 ['난징', '천안문', '드립', '정지', '시발련', '좆', '게임'],
 ['시발', '짱꺠', '씨발', '뒤저', '꺠년'],
 ['기준',
  '신섭',
  '발표',
  '이태',
  '시간',
  '결론',
  '시즌',
  '시작',
  '사람',
  '중',
  '과금',
  '러',
  '빨',
  '신섭',
  '다시',
  '한번',
  '빨',
  '섭',
  '이전',
  '마지막',
  '빨',
  '운영',
  '포기',
 

In [30]:
#토픽모델링에 필요한 Dictionary 와 Corpus 만들기 (no_below = 100 빈도수 100이하 단어 제거)
id2word=corpora.Dictionary(data_word)
id2word.filter_extremes(no_below = 100)
texts = data_word
corpus=[id2word.doc2bow(text) for text in texts]

In [31]:
mallet_path = 'C:\\Users\\sh921\\Desktop\\mallet-2.0.8\\mallet-2.0.8\\bin\\mallet' 
import os
os.environ['MALLET_HOME'] = 'C:\\Users\\sh921\\Desktop\\mallet-2.0.8\\mallet-2.0.8\\'
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
print(ldamallet.show_topics(formatted=False))

[(2, [('게임', 0.5776184072911997), ('이', 0.16696548632900046), ('최고', 0.06835499775885254), ('공포', 0.05871806364858808), ('당신', 0.04915583445390707), ('인생', 0.02300911399970118), ('술래잡기', 0.013745704467353952), ('암', 0.012401015986851935), ('세계', 0.010309278350515464), ('세상', 0.007096966980427312)]), (3, [('무료', 0.12197911107722803), ('데', 0.06350537470458184), ('컨텐츠', 0.05489059998475261), ('스토리', 0.04208279332164367), ('시즌', 0.04147289776625753), ('업데이트', 0.03926202637798277), ('데바', 0.035755126934512466), ('구매', 0.029198749714111457), ('지금', 0.02797895860333918), ('할인', 0.02485324388198521)]), (11, [('게임', 0.26320332731695834), ('꿀잼', 0.05108687122341678), ('스팀', 0.04857568861335635), ('개꿀잼', 0.03947265165188731), ('계속', 0.03860943262967904), ('만', 0.03743231578121321), ('가끔', 0.027936906536922233), ('제일', 0.026995213058149573), ('배그', 0.025975045122812526), ('느낌', 0.022208271207721886)]), (15, [('살인마', 0.18171540548688575), ('매칭', 0.1128278564968345), ('생존자', 0.08901115465782333), (

In [32]:
#lda모델의 일관성 점수 계산
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.4740150026115068


In [33]:
#최적의 토픽 수를 찾기 위해 여러 토픽 수로 일관성을 계산하고 비교
def compute_coherence_values(dictionary, corpus, texts, limit, start=14, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=6, limit=41, step=2)

In [ ]:
# 일관성 점수 그래프 보기
limit=41; start=6; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# 토픽수 별 일관성 점수 확인
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# 토픽 수 선택하고 토픽들(단어) 확인
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=20))

In [ ]:
#트윗들의 토픽을 확인 (토픽별로)
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    sent_topics_df = pd.DataFrame()

    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: 
                wp = ldamodel.show_topic(topic_num,topn=40)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=Data_list)
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])
df_dominant_topic

In [ ]:
# 각 토픽에서 가장 대표적인 문장 찾기
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", "Text", "Num_Documents", "Perc_Documents"]


lda_inform

In [ ]:
lda_inform.to_csv ("LDA_steam.csv", index = None)
lda_inform

LDAvis로 시각화

차원 축소 방법인 Principal Component Analysis (PCA) 와 키워드 추출 방법을 이용하여 토픽 간의 관계와 토픽 키워드를 손쉽게 이해할 수 있도록 도와줌

토픽은 단어 개수의 차원을 가지고 있음->2 차원으로 압축하기 위해서는 차원 축소 방법 이용

LDAvis는 Principal Component Analysis (PCA) 를 이용하여 n_terms 차원의 벡터들을 2 차원으로 압축

-원의 크기 – 토픽의 단어들이 얼마나 속해 있고 어떻게 분포 되어있는지

-원의 거리- 토픽 간의 유사성 ( 두 개의 원이 겹친다면, 두 개의 토픽은 유사한 토픽이라는 의미

-버블 중 하나로 커서를 이동하면 오른쪽의 단어와 막대가 업데이트

-토픽을 형성하게 하는 주도적인 키워드 확인 가능


λ =1 일수록 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미

λ = 0 일수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미(해당 토픽에서 많이 등장한 단어)

In [55]:
from gensim.models.wrappers import LdaMallet

C:\Users\sh921\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ImportError: cannot import name 'Vocab' from 'gensim.models.word2vec' (C:\Users\sh921\anaconda3\lib\site-packages\gensim\models\word2vec.py)

In [56]:
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis

C:\Users\sh921\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: module 'gensim.models' has no attribute 'wrappers'

In [46]:
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(optimal_model)
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, corpus, id2word)
vis

C:\Users\sh921\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: module 'gensim.models' has no attribute 'wrappers'

In [53]:
import pyLDAvis
import pyLDAvis.gensim

model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(optimal_model)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis

C:\Users\sh921\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: No module named 'pyLDAvis.gensim'

In [ ]:
#결과 저장하기
topic_num=str(topic_num)
df_dominant_topic.to_csv ('./'+topic_num+'_topic_tweet.csv', index = None,encoding="euc-kr")
lda_inform.to_csv ('./'+topic_num+'_topic_count.csv', index = None,encoding="euc-kr")
pyLDAvis.save_html(vis, './'+topic_num+'.html')